In [1]:
# process_utils.py
import psutil
import os
import platform

def find_process_by_port(port):
    for proc in psutil.process_iter(['pid', 'name']):
        try:
            connections = proc.connections(kind='inet')
            for conn in connections:
                if conn.laddr.port == port:
                    return proc.pid
        except (psutil.AccessDenied, psutil.NoSuchProcess):
            continue
    return None

def kill_process(pid):
    try:
        if platform.system() == "Windows":
            os.system(f"taskkill /PID {pid} /F")
        else:
            os.system(f"kill -9 {pid}")
        print(f"Process with PID {pid} has been terminated.")
    except Exception as e:
        print(f"Error terminating process with PID {pid}: {e}")

def kill_port_process(port:int):
    pid = find_process_by_port(port)
    
    if pid:
        print(f"Process found using port {port}: PID {pid}")
        kill_process(pid)
    else:
        print(f"No process found using port {port}")
        
pyu_port = 16307
spu_port = 11666

kill_port_process(port=pyu_port)
kill_port_process(port=spu_port)

No process found using port 16307
No process found using port 11666


/tmp/ipykernel_1082924/2512508200.py:9: DeprecationWarning: connections() is deprecated and will be removed; use net_connections() instead
  connections = proc.connections(kind='inet')


In [2]:
import secretflow as sf

# Check the version of your SecretFlow
print("The version of SecretFlow: {}".format(sf.__version__))

# In case you have a running secretflow runtime already.
sf.shutdown()

cluster_config = {
    "parties": {
        "alice": {
            # replace with alice's real address.
            "address": "ecm-01:" + str(pyu_port),
            "listen_addr": "0.0.0.0:" + str(pyu_port),
        },
        "bob": {
            # replace with bob's real address.
            "address": "ecm-02:" + str(pyu_port),
            "listen_addr": "0.0.0.0:" + str(pyu_port),
        },
    },
    "self_party": "bob",
}


tls_config = {
    "ca_cert": "/home/beng003/certificate/alice_ca.crt",
    "cert": "/home/beng003/certificate/bob_server_cert.crt",
    "key": "/home/beng003/certificate/bob_server_key.key",
}


sf.init(address="ecm-02:6379", cluster_config=cluster_config, tls_config=tls_config)
alice = sf.PYU("alice")
bob = sf.PYU("bob")
print("Alice and Bob are ready to go!")

2024-07-30 18:24:11,393	INFO worker.py:1540 -- Connecting to existing Ray cluster at address: ecm-02:6379...
2024-07-30 18:24:11,401	INFO worker.py:1724 -- Connected to Ray cluster.
2024-07-30 18:24:11.424 INFO api.py:233 [bob] -- [Anonymous_job] Started rayfed with {'CLUSTER_ADDRESSES': {'alice': 'ecm-01:16307', 'bob': '0.0.0.0:16307'}, 'CURRENT_PARTY_NAME': 'bob', 'TLS_CONFIG': {'ca_cert': '/home/beng003/certificate/alice_ca.crt', 'cert': '/home/beng003/certificate/bob_server_cert.crt', 'key': '/home/beng003/certificate/bob_server_key.key'}}


The version of SecretFlow: 1.8.0b0


2024-07-30 18:24:12.369 INFO barriers.py:284 [bob] -- [Anonymous_job] Succeeded to create receiver proxy actor.
(ReceiverProxyActor pid=1083217) 2024-07-30 18:24:12.365 INFO grpc_proxy.py:359 [bob] -- [Anonymous_job] ReceiverProxy binding port 16307, options: (('grpc.enable_retries', 1), ('grpc.so_reuseport', 0), ('grpc.max_send_message_length', 524288000), ('grpc.max_receive_message_length', 524288000), ('grpc.service_config', '{"methodConfig": [{"name": [{"service": "GrpcService"}], "retryPolicy": {"maxAttempts": 5, "initialBackoff": "5s", "maxBackoff": "30s", "backoffMultiplier": 2, "retryableStatusCodes": ["UNAVAILABLE"]}}]}'))...
(ReceiverProxyActor pid=1083217) 2024-07-30 18:24:12.368 INFO grpc_proxy.py:379 [bob] -- [Anonymous_job] Successfully start Grpc service with credentials.
2024-07-30 18:24:13.260 INFO barriers.py:333 [bob] -- [Anonymous_job] SenderProxyActor has successfully created.
2024-07-30 18:24:13.261 INFO barriers.py:520 [bob] -- [Anonymous_job] Try ping ['alice'] 

Alice and Bob are ready to go!


(_run pid=1080262) INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': 
(_run pid=1080262) INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=1080262) INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
(SenderProxyActor pid=1083266) 2024-07-30 18:28:45.817 INFO xla_bridge.py:863 [bob] -- [Anonymous_job] Unable to initialize backend 'cuda': 
(SenderProxyActor pid=1083266) 2024-07-30 18:28:45.817 INFO xla_bridge.py:863 [bob] -- [Anonymous_job] Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(SenderProxyActor pid=1083266) 2024-07-30 18:28:45.818 INFO xla_bridge.py:863 [bob] -- [Anonymous_job] Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or d

In [3]:
import spu

cluster_def = {
    "nodes": [
        {
            "party": "alice",
            # The address for other peers.
            "address": "ecm-01:" + str(spu_port),
            # The listen address of this node.
            # Optional. Address will be used if listen_address is empty.
            "listen_address": "0.0.0.0:" + str(spu_port),
        },
        {
            "party": "bob",
            "address": "ecm-02:" + str(spu_port),
            "listen_address": "0.0.0.0:" + str(spu_port),
        },
    ],
    "runtime_config": {
        "protocol": spu.spu_pb2.SEMI2K,
        "field": spu.spu_pb2.FM128,
        "sigmoid_mode": spu.spu_pb2.RuntimeConfig.SIGMOID_REAL,
    },
}

link_desc = {
    "recv_timeout_ms": 6000,
}

spu_device = sf.SPU(cluster_def=cluster_def, link_desc=link_desc)
print("*****************************************************Bob SPU")

*****************************************************Bob SPU


In [4]:
print("*****************************************************Bob PSI")
input_path = {alice: "/home/beng003/python_project/sf-test/data/v_alice.csv", bob: "/home/beng003/python_project/sf-test/data/v_bob.csv"}
output_path = {alice: "/home/beng003/python_project/sf-test/data/alice_psi.csv", bob: "/home/beng003/python_project/sf-test/data/bob_psi.csv"}
# spu_device.psi_csv("uid", input_path, output_path, "alice")
from secretflow.data.vertical import read_csv as v_read_csv

vdf = v_read_csv(input_path, spu=spu_device, keys='uid', drop_keys="uid")

print("*****************************************************Bob PSI")
vdf.to_csv(output_path, index=False)


*****************************************************Bob PSI


2024-07-30 18:25:12.677 INFO proxy.py:187 [bob] -- [Anonymous_job] Create proxy actor <class 'secretflow.device.proxy.ActorPartitionAgent'> with party alice.
2024-07-30 18:25:12.678 INFO proxy.py:187 [bob] -- [Anonymous_job] Create proxy actor <class 'secretflow.device.proxy.ActorPartitionAgent'> with party bob.


(SPURuntime(device_id=None, party=bob) pid=1083331) [2024-07-30 18:25:12.667] [info] [launch.cc:164] LEGACY PSI config: {"psi_type":"KKRT_PSI_2PC","broadcast_result":true,"input_params":{"path":"/home/beng003/python_project/sf-test/data/v_bob.csv","select_fields":["uid"],"precheck":true},"output_params":{"path":"/home/beng003/python_project/sf-test/data/v_bob.csv.psi_output_52218","need_sort":true},"curve_type":"CURVE_25519","bucket_size":1048576}
(SPURuntime(device_id=None, party=bob) pid=1083331) [2024-07-30 18:25:12.667] [info] [bucket_psi.cc:400] bucket size set to 1048576
(SPURuntime(device_id=None, party=bob) pid=1083331) [2024-07-30 18:25:12.668] [info] [bucket_psi.cc:252] Begin sanity check for input file: /home/beng003/python_project/sf-test/data/v_bob.csv, precheck_switch:true
(SPURuntime(device_id=None, party=bob) pid=1083331) [2024-07-30 18:25:12.670] [info] [csv_checker.cc:135] Executing duplicated scripts: LC_ALL=C sort --parallel=8 --buffer-size=1G --stable selected-keys

*****************************************************Bob PSI


In [5]:
sf.reveal(vdf.values.partitions[bob])

array([[  0,  14,   1, 153,   1,  -1,   0,   0,   0],
       [  0,  14,   1, 360,   2,  -1,   0,   0,   0]])

In [12]:
alice_data_pyu = alice.load("/home/beng003/python_project/sf-test/data/alice_data.txt")
alice_data_spu = alice_data_pyu.to(spu_device)
bob_data_pyu = bob.load("/home/beng003/python_project/sf-test/data/bob_data.txt")
bob_data_spu = bob_data_pyu.to(spu_device)


print("数据读取完成")

数据读取完成


In [13]:
sf.reveal(bob_data_pyu)

22

In [16]:
def apply_operator(a, b):
    return a + b


alice_add_bob_data = spu_device(apply_operator)(alice_data_spu, bob_data_spu)
print("计算完成")
print(alice_add_bob_data)
alice_value = alice_add_bob_data.to(alice)

计算完成


KeyboardInterrupt: 

KeyboardInterrupt: 

In [ ]:
alice_add_bob_data_m = sf.reveal(alice_add_bob_data)
print(alice_add_bob_data_m)

In [6]:

sf.shutdown()

KeyboardInterrupt: 